In [1]:
# import dependencies
import pandas as pd
import gmaps
import requests

# import API key
from config import g_key

In [2]:
# store CSV saved and created in part 1 into a dataframe
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tasiilaq,65.6145,-37.6368,30.92,85,98,7.07,GL,2021-09-25 00:40:20
1,1,Cape Town,-33.9258,18.4232,55.27,90,0,1.14,ZA,2021-09-25 00:37:35
2,2,Hilo,19.7297,-155.0900,79.79,85,90,13.80,US,2021-09-25 00:40:37
3,3,Mazatlan,23.2167,-106.4167,85.96,74,20,9.22,MX,2021-09-25 00:40:27
4,4,Chokurdakh,70.6333,147.9167,33.75,75,100,15.30,RU,2021-09-25 00:41:39


In [3]:
# get data types
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# configures gmaps to use api key
gmaps.configure(api_key=g_key)

In [5]:
# get maximum temperature
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=0.01)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# ask customer to add min and max temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# filter dataset to find cities fitting criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Hilo,19.7297,-155.0900,79.79,85,90,13.80,US,2021-09-25 00:40:37
3,3,Mazatlan,23.2167,-106.4167,85.96,74,20,9.22,MX,2021-09-25 00:40:27
13,13,Georgetown,5.4112,100.3354,83.89,90,20,1.99,MY,2021-09-25 00:40:41
20,20,Sao Jose Da Coroa Grande,-8.8978,-35.1478,75.49,86,33,10.60,BR,2021-09-25 00:41:44
23,23,Roebourne,-20.7833,117.1333,79.75,24,5,11.90,AU,2021-09-25 00:41:45
25,25,Filingue,14.3521,3.3168,85.50,55,92,11.63,NE,2021-09-25 00:41:45
30,30,Asyut,27.1810,31.1837,75.38,45,0,10.60,EG,2021-09-25 00:41:47
33,33,West Bay,19.3667,-81.4167,84.07,80,100,8.19,KY,2021-09-25 00:41:48
40,40,San Patricio,28.0170,-97.5169,82.26,54,86,8.93,US,2021-09-25 00:41:37
41,41,Kavieng,-2.5744,150.7967,84.74,67,21,10.40,PG,2021-09-25 00:41:51


In [11]:
preferred_cities_df.count()

City_ID       172
City          172
Lat           172
Lng           172
Max Temp      172
Humidity      172
Cloudiness    172
Wind Speed    172
Country       172
Date          172
dtype: int64

In [12]:
# create DataFrame to store hotel names along with city, country, max temp, lat, and lng
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Hilo,US,79.79,19.7297,-155.0900,
3,Mazatlan,MX,85.96,23.2167,-106.4167,
13,Georgetown,MY,83.89,5.4112,100.3354,
20,Sao Jose Da Coroa Grande,BR,75.49,-8.8978,-35.1478,
23,Roebourne,AU,79.75,-20.7833,117.1333,
25,Filingue,NE,85.50,14.3521,3.3168,
30,Asyut,EG,75.38,27.1810,31.1837,
33,West Bay,KY,84.07,19.3667,-81.4167,
40,San Patricio,US,82.26,28.0170,-97.5169,
41,Kavieng,PG,84.74,-2.5744,150.7967,


In [13]:
# set parameters to search for hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# iterate through dataframe
for index, row in hotel_df.iterrows():
    # get lat and lng
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add latitude and longitude to location key for the params dictionary
    params["location"] = f'{lat}{lng}'
    
    # use search term: lodging and latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #make request and get JSON data from search
    hotels = requests.get(base_url, params=params).json()
    
    # grab first hotel from results and store name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping")

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... s

In [17]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]